In [ ]:
# 1. Gerekli Kütüphaneler ve Kurulumlar
!pip install torch==2.3.1 torchtext==0.18.0
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm
!pip install datasets

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset
from torchtext.vocab import build_vocab_from_iterator
import spacy
import math
import random
import numpy as np

# Device settings (once again we must pick a GPU if avaliable)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# Stabilize Randomness
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Preperation of data and Tokenizer
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)] # Transformer has no inherent direction

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

dataset = load_dataset("bentrevett/multi30k")

def yield_tokens(data_iter, language):
    for example in data_iter:
        if language == 'de':
            yield tokenize_de(example['de'])
        else:
            yield tokenize_en(example['en'])

special_symbols = ['<unk>', '<pad>', '<sos>', '<eos>']
vocab_de = build_vocab_from_iterator(yield_tokens(dataset['train'], 'de'), min_freq=2, specials=special_symbols)
vocab_en = build_vocab_from_iterator(yield_tokens(dataset['train'], 'en'), min_freq=2, specials=special_symbols)

vocab_de.set_default_index(vocab_de['<unk>'])
vocab_en.set_default_index(vocab_en['<unk>'])

PAD_IDX = vocab_de['<pad>']
SOS_IDX = vocab_de['<sos>']
EOS_IDX = vocab_de['<eos>']
BATCH_SIZE = 32 # Transformer consumes too much memory

# Batch Preperation
def collate_batch(batch):
    src_list, tgt_list = [], []
    for item in batch:
        src_tensor = torch.tensor([SOS_IDX] + [vocab_de[token] for token in tokenize_de(item['de'])] + [EOS_IDX], dtype=torch.long)
        tgt_tensor = torch.tensor([SOS_IDX] + [vocab_en[token] for token in tokenize_en(item['en'])] + [EOS_IDX], dtype=torch.long)
        src_list.append(src_tensor)
        tgt_list.append(tgt_tensor)

    src_batch = pad_sequence(src_list, padding_value=PAD_IDX, batch_first=True) # Transformer batch_first=True diğerini de dene
    tgt_batch = pad_sequence(tgt_list, padding_value=PAD_IDX, batch_first=True)
    return src_batch.to(device), tgt_batch.to(device)

train_loader = DataLoader(dataset['train'], batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(dataset['validation'], batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)
test_loader = DataLoader(dataset['test'], batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 873.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

/usr/local/lib/python3.12/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.12/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


Cihaz: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

...

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/29000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1014 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Veri hazırlığı tamamlandı!


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout: float, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(0)

        self.dropout = nn.Dropout(dropout)
        # save as buffer 
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding):
        # Add position information to word vector
        return self.dropout(token_embedding + self.pos_embedding[:, :token_embedding.size(1), :])

class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers, num_decoder_layers, emb_size, nhead, src_vocab_size, tgt_vocab_size, dim_feedforward, dropout=0.1):
        super(Seq2SeqTransformer, self).__init__()

        # Embedding
        self.src_tok_emb = nn.Embedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = nn.Embedding(tgt_vocab_size, emb_size)

        # Position information
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

        # Main Transformer block (Encoder + Decoder)
        self.transformer = nn.Transformer(d_model=emb_size,
                                          nhead=nhead,
                                          num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers,
                                          dim_feedforward=dim_feedforward,
                                          dropout=dropout,
                                          batch_first=True) # Data in (Batch, Seq) format

        # Output layer (Word prediction)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)

    def forward(self, src, trg, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask):
        # Embedding + Positional Encoding
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))

        # Flow through Transformer
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)

        return self.generator(outs)

    def encode(self, src, src_mask):
        return self.transformer.encoder(self.positional_encoding(self.src_tok_emb(src)), src_mask)

    def decode(self, tgt, memory, tgt_mask):
        return self.transformer.decoder(self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask)

In [ ]:
def generate_square_subsequent_mask(sz):
    # Masking 
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[1]
    tgt_seq_len = tgt.shape[1]

    # Target mask 
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=device).type(torch.bool) # Encoder can see everything

    # Padding mask (Don't see spaces)
    src_padding_mask = (src == PAD_IDX)
    tgt_padding_mask = (tgt == PAD_IDX)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
# Hyperparametrs
SRC_VOCAB_SIZE = len(vocab_de)
TGT_VOCAB_SIZE = len(vocab_en)
EMB_SIZE = 512
NHEAD = 8               # Num of attention heads 
FFN_HID_DIM = 512       # Feed Forward layer dimension
BATCH_SIZE = 32
NUM_ENCODER_LAYERS = 3  # Change here for Ablation (3-e)
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

# Use Xavier Initialization for weights for faster learning
for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

print(f"Model Parametre Count: {sum(p.numel() for p in transformer.parameters() if p.requires_grad)}")


Model Parametre Sayısı: 23073839


In [ ]:
import time
import torch.optim as optim
from torch.utils.data import Subset

# --- HYPERPARAMETRS ---
DATA_LIMIT = 1500         # DATA_LIMIT must be increased
N_EPOCHS = 2              # Epoch must be at least 5
LEARNING_RATE = 0.0005

# Model Architecture
# Dimension were chosen so that it can be compared with RNN
ENC_LAYERS = 2            # Num of Encoder layers
DEC_LAYERS = 2            # Num of Decoder layers
EMB_SIZE = 256            # Embedding layers
NHEAD = 4                 # num of Attention heads
FFN_HID_DIM = 256         # Feedforward layer dimension

# Dataset Preperation
train_subset = Subset(dataset['train'], range(DATA_LIMIT))
val_subset = Subset(dataset['validation'], range(500))

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, collate_fn=collate_batch)

print(f"Training set dimension: {len(train_subset)} sentence")

# Setting up Model
transformer_model = Seq2SeqTransformer(
    num_encoder_layers=ENC_LAYERS,
    num_decoder_layers=DEC_LAYERS,
    emb_size=EMB_SIZE,
    nhead=NHEAD,
    src_vocab_size=len(vocab_de),
    tgt_vocab_size=len(vocab_en),
    dim_feedforward=FFN_HID_DIM
).to(device)

# Xavier Initialization for weights
for p in transformer_model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

# Optimizer
optimizer = optim.Adam(transformer_model.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.98), eps=1e-9)
loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

print(f"Number of Transformer Model Parametres: {sum(p.numel() for p in transformer_model.parameters() if p.requires_grad):,}")

# Standard Training Function
def train_epoch(model, iterator, optimizer, criterion, device):
    model.train()
    epoch_loss = 0

    for src, tgt in iterator:
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:, :-1] # Final token except for (<eos>) 

        # Create Masks
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        # Run the Model
        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[:, 1:] # First token except for (<sos>)
        loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

# Standard evaluation function
def evaluate_epoch(model, iterator, criterion, device):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for src, tgt in iterator:
            src = src.to(device)
            tgt = tgt.to(device)

            tgt_input = tgt[:, :-1]
            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

            logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

            tgt_out = tgt[:, 1:]
            loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

# --- Training Loop ---
for epoch in range(1, N_EPOCHS+1):
    start_time = time.time()

    train_loss = train_epoch(transformer_model, train_loader, optimizer, loss_fn, device)
    val_loss = evaluate_epoch(transformer_model, val_loader, loss_fn, device)

    end_time = time.time()

    print(f"Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Val Loss: {val_loss:.3f} | Süre: {(end_time - start_time):.1f}s")

Eğitim Seti Boyutu: 1500 cümle
Transformer Modeli Hazır. Parametre Sayısı: 7,339,055

--- Model Eğitimi Başlıyor (2 Epoch) ---


/usr/local/lib/python3.12/dist-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch: 01 | Train Loss: 6.282 | Val Loss: 5.241 | Süre: 39.3s
Epoch: 02 | Train Loss: 5.008 | Val Loss: 4.938 | Süre: 29.7s
Eğitim Tamamlandı.


In [ ]:
from torchtext.data.metrics import bleu_score

# Greedy Decoding (batch_first=True)
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    # src: [1, seq_len] (Batch size 1, Row length N)
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)

    # Starting token (Batch=1, Seq=1)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)

    for i in range(max_len-1):
        memory = memory.to(device)

        # ys.size(1) is the length of sequence 
        tgt_mask = (generate_square_subsequent_mask(ys.size(1))
                    .type(torch.bool)).to(device)

        out = model.decode(ys, memory, tgt_mask)

        # no need to transpose since batch_first=True  
        # Take the probabilities of final words: out[:, -1] -> [Batch, Dim]
        prob = model.generator(out[:, -1])

        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)

        if next_word == EOS_IDX:
            break

    return ys

# Translation Function
def translate_sentence_transformer(model, sentence, vocab_de, vocab_en):
    model.eval()

    # Tokenize and convert to ID
    # unsqueeze(0) -> [1, Seq_Len]
    src = torch.tensor([vocab_de[token] for token in tokenize_de(sentence)]).long().unsqueeze(0).to(device)
    num_tokens = src.shape[1]

    # Encoder mask
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool).to(device)

    # Predict
    tgt_tokens = greedy_decode(model, src, src_mask, max_len=num_tokens + 5, start_symbol=SOS_IDX).flatten()

    # Convert to word
    return [vocab_en.lookup_token(i) for i in list(tgt_tokens.cpu().numpy())]

# BLEU Calculation
def calculate_bleu_transformer(data, model):
    trgs = []
    pred_trgs = []

    # Calculate for first 100 examples of Test set (To speed up)
    for i, datum in enumerate(data):
        if i > 100: break
        src = datum['de']
        trg = datum['en']
        pred_tokens = translate_sentence_transformer(model, src, vocab_de, vocab_en)
        pred_str = [t for t in pred_tokens if t not in ['<sos>', '<eos>', '<pad>']]
        trg_tokens = tokenize_en(trg)
        pred_trgs.append(pred_str)
        trgs.append([trg_tokens])

    return bleu_score(pred_trgs, trgs)

In [ ]:
# An example translation
print("--- Example Translation ---")
test_sentence = "Ein Mann spielt Fußball."
translation = translate_sentence_transformer(transformer_model, test_sentence, vocab_de, vocab_en)
print(f"Source: {test_sentence}")
print(f"Translation: {' '.join(translation).replace('<sos>', '').replace('<eos>', '')}")

# BLEU Score calculation
score = calculate_bleu_transformer(dataset['test'], transformer_model)
print(f"Model BLEU Score: {score*100:.2f}")

--- Örnek Çeviri ---
Kaynak: Ein Mann spielt Fußball.
Çeviri:  A man in a a a . 
Model BLEU Skoru: 2.71
